In [13]:
import pandas as pb

data = pb.read_excel('FAKSexport.xlsx')
#print(data)
print(data.shape)



Exception ignored in: <function ZipFile.__del__ at 0x00000210F9E8BB00>
Traceback (most recent call last):
  File "C:\Users\gp4l\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1929, in __del__
    self.close()
  File "C:\Users\gp4l\AppData\Local\Programs\Python\Python312\Lib\zipfile\__init__.py", line 1946, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


(129730, 708)


In [14]:
import pandas as pb

Smaldata = pb.read_excel('SmalFAKS.xlsx')
#print(data)
print(Smaldata.shape)


(2, 708)


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Konvertera alla kolumnnamn till strängar för att undvika TypeError
Smaldata.columns = Smaldata.columns.astype(str)
data.columns = data.columns.astype(str)

print("1.6")
# Sök efter kolumner som innehåller det specifika ordet
word_to_remove = 'checkbox'
columns_to_remove = [col for col in Smaldata.columns if word_to_remove in col]

print(Smaldata.shape)
print(data.shape)
print("2")
# Ta bort de hittade kolumnerna från DataFrame
data = data.drop(columns=columns_to_remove)

print(Smaldata.shape)
print(data.shape)
print("3")
# Spara den uppdaterade DataFrame till en ny Excel-fil eller samma fil
data.to_excel('updFAKSexp.xlsx', index=False)

1.6
(2, 708)
(129730, 708)
2
(2, 708)
(129730, 487)
3


In [68]:
import pandas as pb
data = pb.read_excel('updFAKSexp.xlsx')
print(1)
data.to_csv('updFAKSexpCSV.csv')
data_np = data.to_numpy()
print(data.shape)

1
(129730, 487)


In [70]:
orgdata = data

In [84]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import numpy as np

model_name = "AI-Sweden-Models/gpt-sw3-1.3b"

#data = pb.read_excel('updFAKSexp.xlsx')
print("1.6")


data = data.astype(str)

print(data.iloc[:, 0])
print(data.iloc[:, 1:])

# Tokenisera texten med en tokeniserare
tokenizer = AutoTokenizer.from_pretrained(model_name)

input_tokens = [tokenizer.encode(input_text) for input_text in data.iloc[:, 2:4].values]
output_tokens = [tokenizer.encode(output_text) for output_text in data.iloc[:, 0].values]



par_tokens = [(input_tokens[i], output_tokens[i])for i in range(len(input_tokens))]

print(par_tokens.shape)

print("1.6")

# Dela upp datamängden enligt 80-10-10-split
train_texts, val_and_test_texts = train_test_split(par_tokens, test_size=0.2, random_state=42) #måste läsa lite om random_state
val_texts, test_texts = train_test_split(val_and_test_texts, test_size=0.5, random_state=42)
print("1.6")


print(test_texts)
print(train_texts)

# Spara de delade datamängderna till separata filer
with open('train_data.txt', 'w') as f:
    for text in train_texts:
        f.write(' '.join(text) + '\n')

with open('val_data.txt', 'w') as f:
    for text in val_texts:
        f.write(' '.join(text) + '\n')

with open('test_data.txt', 'w') as f:
    for text in test_texts:
        f.write(' '.join(text) + '\n')

1.6


AttributeError: 'list' object has no attribute 'values'